In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# ----------------------------------------------------------
# 1. Load Dataset
# ----------------------------------------------------------
df_red = pd.read_csv("/Users/animeshdash/Downloads/wine+quality/winequality-red.csv", sep=";")
df_white = pd.read_csv("/Users/animeshdash/Downloads/wine+quality/winequality-white.csv", sep=";")

df = pd.concat([df_red, df_white], axis=0)

X = df.drop("quality", axis=1).values
y = df["quality"].values.reshape(-1, 1)

In [3]:
# ----------------------------------------------------------
# 2. Train–test split + scaling
# ----------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
# ----------------------------------------------------------
# 3. Network Architecture (2 Hidden Layers)
# ----------------------------------------------------------
input_dim = X_train.shape[1]   # 11
hidden_dim1 = 64
hidden_dim2 = 32
output_dim = 1                # regression

# Weight initialization
W1 = np.random.randn(input_dim, hidden_dim1) * 0.01
b1 = np.zeros((1, hidden_dim1))

W2 = np.random.randn(hidden_dim1, hidden_dim2) * 0.01
b2 = np.zeros((1, hidden_dim2))

W3 = np.random.randn(hidden_dim2, output_dim) * 0.01
b3 = np.zeros((1, output_dim))


In [5]:
# ----------------------------------------------------------
# 4. Activation Functions
# ----------------------------------------------------------
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

In [6]:
# ----------------------------------------------------------
# 5. Forward Pass
# ----------------------------------------------------------
def forward_pass(X):
    z1 = X @ W1 + b1
    a1 = relu(z1)

    z2 = a1 @ W2 + b2
    a2 = relu(z2)

    z3 = a2 @ W3 + b3  # linear output
    return z1, a1, z2, a2, z3


In [7]:
# ----------------------------------------------------------
# 6. Loss Function
# ----------------------------------------------------------
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)


In [8]:
# ----------------------------------------------------------
# 7. Backpropagation
# ----------------------------------------------------------
def backward_pass(X, y, z1, a1, z2, a2, y_pred, lr=0.001):
    global W1, b1, W2, b2, W3, b3

    m = X.shape[0]

    # ---- Output layer ----
    d_y_pred = (2 / m) * (y_pred - y)

    dW3 = a2.T @ d_y_pred
    db3 = np.sum(d_y_pred, axis=0, keepdims=True)

    # ---- Hidden layer 2 ----
    d_hidden2 = d_y_pred @ W3.T * relu_derivative(z2)

    dW2 = a1.T @ d_hidden2
    db2 = np.sum(d_hidden2, axis=0, keepdims=True)

    # ---- Hidden layer 1 ----
    d_hidden1 = d_hidden2 @ W2.T * relu_derivative(z1)

    dW1 = X.T @ d_hidden1
    db1 = np.sum(d_hidden1, axis=0, keepdims=True)

    # ---- Gradient descent update ----
    W3 -= lr * dW3
    b3 -= lr * db3

    W2 -= lr * dW2
    b2 -= lr * db2

    W1 -= lr * dW1
    b1 -= lr * db1


In [9]:
# ----------------------------------------------------------
# 8. Training Loop
# ----------------------------------------------------------
epochs = 4000
lr = 0.05

for epoch in range(epochs):
    z1, a1, z2, a2, y_pred = forward_pass(X_train)
    loss = mse_loss(y_pred, y_train)

    backward_pass(X_train, y_train, z1, a1, z2, a2, y_pred, lr)

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Train MSE = {loss:.4f}")


Epoch 0, Train MSE = 34.5774
Epoch 500, Train MSE = 0.5077
Epoch 1000, Train MSE = 0.4830
Epoch 1500, Train MSE = 0.4626
Epoch 2000, Train MSE = 0.4479
Epoch 2500, Train MSE = 0.4351
Epoch 3000, Train MSE = 0.4391
Epoch 3500, Train MSE = 0.4325


In [10]:
# ----------------------------------------------------------
# 9. Test Evaluation
# ----------------------------------------------------------
_, _, _, _, test_pred = forward_pass(X_test)
test_loss = mse_loss(test_pred, y_test)
print("\nFinal Test MSE:", test_loss)


Final Test MSE: 0.48613293792622575


In [11]:
# ----------------------------------------------------------
# 10. Predictions vs Actual
# ----------------------------------------------------------
print("\nSample Predictions vs Actual:")
for i in range(10):
    predicted = float(test_pred[i])
    actual = float(y_test[i])
    print(f"Predicted: {predicted:.2f}   |   Actual: {actual}")


Sample Predictions vs Actual:
Predicted: 6.26   |   Actual: 8.0
Predicted: 4.96   |   Actual: 5.0
Predicted: 6.96   |   Actual: 7.0
Predicted: 5.51   |   Actual: 6.0
Predicted: 5.25   |   Actual: 6.0
Predicted: 6.34   |   Actual: 6.0
Predicted: 5.38   |   Actual: 5.0
Predicted: 6.34   |   Actual: 6.0
Predicted: 4.81   |   Actual: 5.0
Predicted: 6.40   |   Actual: 7.0


/var/folders/3b/jw5gpnhd34q471085qb1b1mc0000gn/T/ipykernel_37076/1768496122.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted = float(test_pred[i])
/var/folders/3b/jw5gpnhd34q471085qb1b1mc0000gn/T/ipykernel_37076/1768496122.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  actual = float(y_test[i])
